In [39]:
import wandb
from argparse import Namespace
import openpyxl

In [2]:
api = wandb.Api(timeout=90)

In [3]:
ds = ['IIIT_WVU', 'NotreDame', 'Clarkson']

In [4]:
from collections import defaultdict

runs = {d:defaultdict(list) for d in ds}

for train in ds:
    for run in api.runs(f"{train}"):
        runs[train][run.config['CLASSIFIER']['model']['type'].replace('torchvision.models.', "")].append(run)

In [5]:
params = {d: {dd: {'bs': 0, 'lambda': 0.0} for dd in ds} for d in ds}

params['IIIT_WVU']['IIIT_WVU'] = {'bs': 64, 'lambda': 0.25}
params['IIIT_WVU']['NotreDame'] = {'bs': 128, 'lambda': 0.75}
params['IIIT_WVU']['Clarkson'] = {'bs': 512, 'lambda': 1.0}

params['NotreDame']['IIIT_WVU'] = {'bs': 512, 'lambda': 0.50}
params['NotreDame']['NotreDame'] = {'bs': 128, 'lambda': 0.25}
params['NotreDame']['Clarkson'] = {'bs': 512, 'lambda': 0.25}

params['Clarkson']['IIIT_WVU'] = {'bs': 256, 'lambda': 2.0}
params['Clarkson']['NotreDame'] = {'bs': 256, 'lambda': 0.25}
params['Clarkson']['Clarkson'] = {'bs': 128, 'lambda': 2.0}

In [6]:
models=sorted(list(set(runs['NotreDame'].keys()).union(set(runs['IIIT_WVU'].keys()),set(runs['Clarkson'].keys()))))

In [36]:
results = {model:{train:{test:Namespace(ACER=0, APCER=0, BPCER=0, invoked=False, run=None) for test in ds} for train in ds} for model in models}

for train in ds:
    for model in runs[train]:
        for run in runs[train][model]:
            batch_size = run.config['CLASSIFIER']['equiv_batch_size']
            lambda_nce = run.config['CUT']['lambda_NCE']
            for test, args in params[train].items():
                if args['bs'] == batch_size and args['lambda'] == lambda_nce:
                    if not results[model][train][test].invoked:
                        results[model][train][test].run = run
                        results[model][train][test].invoked = True
                        results[model][train][test].ACER = run.summary[f"acer/{test}/TEST"]['min']
#                     else:
#                         results[model][train][test].ACER = min(results[model][train][test], run.summary[f"acer/{test}/TEST"]['min'])


for model in models:
    for train in ds:
        for test in ds:
            if results[model][train][test].invoked:
                run = results[model][train][test].run
                for info in run.scan_history(keys=[f"apcer/{test}/TEST", f"bpcer/{test}/TEST", f"acer/{test}/TEST"]):
                    apcer = info[f"apcer/{test}/TEST"]
                    acer = info[f"acer/{test}/TEST"]
                    bpcer = info[f"bpcer/{test}/TEST"]
                    if abs(acer-results[model][train][test].ACER)/acer < 1.0e-5:
                        results[model][train][test].APCER = apcer
                        results[model][train][test].BPCER = bpcer
                
                results[model][train][test].ACER = "{:.02f}".format(results[model][train][test].ACER*100)
                results[model][train][test].APCER = "{:.02f}".format(results[model][train][test].APCER*100)
                results[model][train][test].BPCER = "{:.02f}".format(results[model][train][test].BPCER*100)
            else:
                results[model][train][test].ACER = "-"
                results[model][train][test].APCER = "-"
                results[model][train][test].BPCER = "-"

In [42]:
wb = openpyxl.Workbook()

sheet = wb.create_sheet('ACER')
sheet.append("Train,IIIT_WVU,,,NotreDame,,,Clarkson,,,\n".strip().split(','))
sheet.append("Test,IIIT_WVU,NotreDame,Clarkson,IIIT_WVU,NotreDame,Clarkson,IIIT_WVU,NotreDame,Clarkson\n".strip().split(','))
for model in models:
    msg = f"{model},"
    for train in ds:
        for test in ds:
            msg += f"{results[model][train][test].ACER},"
    sheet.append((msg + "\n").strip().split(','))
    
sheet = wb.create_sheet('APCER')
sheet.append("Train,IIIT_WVU,,,NotreDame,,,Clarkson,,,\n".strip().split(','))
sheet.append("Test,IIIT_WVU,NotreDame,Clarkson,IIIT_WVU,NotreDame,Clarkson,IIIT_WVU,NotreDame,Clarkson\n".strip().split(','))
for model in models:
    msg = f"{model},"
    for train in ds:
        for test in ds:
            msg += f"{results[model][train][test].APCER},"
    sheet.append((msg + "\n").strip().split(','))

sheet = wb.create_sheet('BPCER')
sheet.append("Train,IIIT_WVU,,,NotreDame,,,Clarkson,,,\n".strip().split(','))
sheet.append("Test,IIIT_WVU,NotreDame,Clarkson,IIIT_WVU,NotreDame,Clarkson,IIIT_WVU,NotreDame,Clarkson\n".strip().split(','))
for model in models:
    msg = f"{model},"
    for train in ds:
        for test in ds:
            msg += f"{results[model][train][test].BPCER},"
    sheet.append((msg + "\n").strip().split(','))
    
wb.save('LivDet-iris-2017.xlsx')